In [ ]:
import sklearn

In [ ]:
# import TfidfVectorizer and CountVectorizer from sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
# processed_ds (from preprocessing notebook)
dataset = processed_ds
documents = dataset.data

In [ ]:
type(documents)
documents[0]

# **TF-IDF**

In [ ]:
# set number of features (output vocabulary)
no_features = 100

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'no_features' is a variable you've defined elsewhere with the desired number of features
no_features = 1000  # Example value; adjust as needed

tfidf_vectorizer = TfidfVectorizer(max_df=0.95,  # documents with a term frequency higher than this will be ignored
                                   min_df=2,     # terms must appear in at least 2 documents to be considered
                                   max_features=no_features,  # the size of the output vocabulary
                                   stop_words='english')  # remove English stop words

# To use the vectorizer, you need to provide it with a dataset to fit and transform
# Example:
# dataset = ["document one text", "document two text", ...]
# tfidf_matrix = tfidf_vectorizer.fit_transform(dataset)

In [ ]:
# fit and transform tfidf_vectorizer with dataset
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
# This tfidf_matrix can then be used as input for various machine learning models.

In [ ]:
# inspect features
tfidf_vectorizer.get_feature_names_out()

# **NMF**

In [ ]:
no_topics = 100

In [ ]:
# Number of topics/components
n_components = no_topics

# Instantiate and fit_transform data with NMF
nmf = NMF(n_components=n_components, random_state=1, init='nndsvd')
W = nmf.fit_transform(tfidf_matrix)  # W matrix contains the document-topic distributions
H = nmf.components_  # H matrix contains the topic-term distributions

# Now, W and H can be used for further analysis, like identifying dominant topics for documents

In [ ]:
len(H)

In [ ]:
# Assuming 'nmf' is your fitted NMF model from the previous example
# and 'tfidf_vectorizer' is the TfidfVectorizer you used to transform your documents

feature_names = tfidf_vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(nmf.components_):
    print(f"Topic #{topic_idx + 1}:")

    # Sort the weights in the topic and get the indices of the top 10 features
    top_feature_indices = topic.argsort()[-10:][::-1]

    # Map the indices to actual words
    top_features = [feature_names[i] for i in top_feature_indices]
    print(" ".join(top_features))